# Introduction

In this assignment, you will build a language identification classifier that distinguishes between six languages:

- [Hausa](https://en.wikipedia.org/wiki/Hausa_language)
- [Indonesian](https://en.wikipedia.org/wiki/Indonesian_language)
- [Manobo](https://en.wikipedia.org/wiki/Manobo_languages)
- [Nahuatl](https://en.wikipedia.org/wiki/Nahuatl)
- [Swahili](https://en.wikipedia.org/wiki/Swahili_language)
- [Tagalog](https://en.wikipedia.org/wiki/Tagalog_language)

Some languages can be distinguished easily, because they use different scripts. These six languages, however, use the same ([Latin](https://en.wikipedia.org/wiki/Latin_script)) script with minimal [diacritics](https://en.wikipedia.org/wiki/Diacritic) so it is difficult to hand-craft classifiers based on the presence or absence of particular characters. Indeed, unless you have linguistic training or familiarity with the languages, it is difficult to tell them apart.

How can they be distinguished? A naïve approach is to use word counts as unigram features. However, the number of possible words in a large corpus of five languages is vast. It is essential to look at something smaller — characters.

Even though the six languages use roughly the same characters, the relative frequencies of these characters vary greatly. Thus, using characters as features (unigram character models) is appealing (and fairly effective). It is also true that languages very in their *phonotactics*, the way in which consonants and vowels combine in sequence. Thus, looking at character ngrams (for small values of $n$) is also appealing (and effective). Note, however, that as the value of $n$ increases, this approach runs into the same problem as the word unigram model (sparcity). In this scenario, the model is likely to overfit.

Various kinds of classifiers can be used for this application. NB classifiers, for example, are quite effective. However, inference is slow and performance, given the same training set, is likely to be worse than other options. Simple logistic regression cannot be used because this is an n-way (multinomial) classification problem. Multinomial Logistic Regression (Softmax Regression) is a good fit. 

## Summary

You will perform the following tasks:

1. Implement a training loop for Multinomial Logistic Regression.
2. Implement inference for Multinomal Logistic Regression
3. Determine the optimal order of $n$ for ngrams for MNLR trained on the training set.
4. Calculate and display a confusion matrix for a trigram model evaluated on the test set.
5. Inspect the feature weights, and display the most predictive features for each language. 

# Imports
Do not change.

In [12]:
import csv
import random

import numpy as np
import numpy.typing as npt
import numpy.testing as testing

from collections import Counter

# Helper Functions

## Metrics for Binary Classification

In [13]:
def precision(tp: int, fp: int) -> float:
    """Computes the precision, given true positives and false positives."""
    return tp / (tp + fp)


def recall(tp: int, fn: int) -> float:
    """Computes the recall, given the true positives and false negatives."""
    return tp / (tp + fn)


def f_measure(beta: float, tp: int, fp: int, fn: int) -> float:
    """Computes the F-measure for a given beta, true positives, false positives, and false negatives."""
    return (
        (1 + beta**2)
        * (precision(tp, fp) * recall(tp, fn))
        / (beta**2 * precision(tp, fp) * recall(tp, fn))
    )


def f1(tp: int, fp: int, fn: int) -> float:
    """Computes the F1 measure for a given TP, FP, and FN."""
    return f_measure(1, tp, fp, fn)

## Micro-Averaged Metrics

In [14]:
def micro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes micro-averaged precision."""
    tp_sum = sum(tp.values())
    fp_sum = sum(fp.values())
    return tp_sum / (tp_sum + fp_sum)


def micro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged recall."""
    tp_sum = sum(tp.values())
    fn_sum = sum(fn.values())
    return tp_sum / (tp_sum + fn_sum)


def micro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes micro-averaged F1."""
    mp = micro_precision(tp, fp)
    mr = micro_recall(tp, fn)
    return 2 * (mp * mr) / (mp + mr)

## Macro-Averaged Metrics

In [15]:
def macro_precision(tp: "dict[str, int]", fp: "dict[str, int]") -> float:
    """Computes macro-averaged precision."""
    n = len(tp)
    return (1 / n) * sum([precision(tp[c], fp[c]) for c in tp.keys()])


def macro_recall(tp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged recall."""
    n = len(tp)
    return (1 / n) * sum([recall(tp[c], fn[c]) for c in tp.keys()])


def macro_f1(tp: "dict[str, int]", fp: "dict[str, int]", fn: "dict[str, int]") -> float:
    """Computes macro-averaged F1."""
    n = len(tp)
    return (
        2
        * (macro_precision(tp, fp) * macro_recall(tp, fn))
        / (macro_precision(tp, fp) + macro_recall(tp, fn))
    )

# Preprocess Data

## Loading data from files

We first need to load data from the provided TSV files. Each file is two columns, the language of the document and the document text, separated by a tab (`\t`) character. We load this data into a list of tuples, to maintain the coupling between each document and its label.

In [16]:
def load_data(data_filename: str) -> list[tuple[str, str]]:
    with open(data_filename) as fin:
        reader = csv.reader(fin, delimiter="\t")
        language_document_tuples = [(lang, doc) for (lang, doc) in reader]
    return language_document_tuples

## Feature Extraction

We will use ngrams as features, so we need to be able to extract them.

In [17]:
def extract_ngrams(x: str, n=3) -> "list[str]":
    """Given a string, return all character ngrams of order `n`."""
    return ["".join(s) for s in (zip(*[x[i:] for i in range(n)]))]

## Language Codes to One-Hot Vectors
And we need a function to convert a language code into a **one-hot vector** (called $\mathbf{y}$).

In [18]:
def to_onehot_vector(lang: str, langs: list[str]) -> np.ndarray:
    y = np.zeros(len(langs))
    y[langs.index(lang)] = 1
    return y

We need to be able to convert `dict`s of ngram counts to vectors of ngram counts (using a map from ngrams to dimensions of the vectors).

In [19]:
def vectorize_ngrams(counter: dict[str, int], feature_map: dict[str, int]) -> np.ndarray:
    """
    Given a dict of ngram counts and a map from features to indices, returns a vector of ngram counts.

    Parameters
    ----------
    counter : dict
        Counter/dict of ngram counts
    feature_map : dict
        Map from ngrams to indices

    Returns
    -------
    np.ndarray
        A vector of ngram counts
    """
    feature_vector = np.zeros(len(feature_map))
    for ngram, count in counter.items():
        if ngram in feature_map:
            feature_vector[feature_map[ngram]] = count
    return feature_vector

And putting together the conversion from text into ngrams, and vectorizing the ngrams:

In [20]:
def vectorize_document(document: str, feature_map: dict[str, int], ngram_length: int) -> np.ndarray:
    document_ngrams = extract_ngrams(document, ngram_length)
    vector = vectorize_ngrams(Counter(document_ngrams), feature_map)
    return vector

We need separate functions for preprocessing the training observations and the dev/test observations. The former function must return a map from language names to labels as one-hot vectors as well as a map from features (ngrams) to indices of vector dimensions.

In [21]:
def preprocess_training_observations(
    training_observations: list[tuple[str, str]], n: int = 1
) -> tuple[list[tuple[np.ndarray, np.ndarray]], dict[str, np.ndarray], dict[str, int]]:
    langs = set()
    features = set()
    obs = []

    for lang, doc in training_observations:
        langs.add(lang)
        ngrams = extract_ngrams(doc, n)
        features = features | set(ngrams)
        obs.append((lang, ngrams))
    feature_map = {feature: idx for idx, feature in enumerate(sorted(features))}
    lang_list = list(sorted(langs))
    lang_map = {lang: to_onehot_vector(lang, lang_list) for lang in lang_list}

    obs = [
        (lang_map[lang], vectorize_ngrams(Counter(ngrams), feature_map)) for (lang, ngrams) in obs
    ]

    print(f"{len(obs)} training observations.")
    return obs, lang_map, feature_map

The function for preprocessing test observations (and dev observations) takes the feature map and the language map as arguments and returns only the list of labeled observations.

In [22]:
def preprocess_test_observations(
    test_observations, feature_map: dict[str, int], lang_map: dict[str, np.ndarray], n: int = 1
) -> tuple[list[np.ndarray], list[np.ndarray]]:
    obs = []

    for i, (lang, doc) in enumerate(test_observations):
        vectorized_doc = vectorize_document(doc, feature_map, ngram_length=n)
        try:
            obs.append((lang_map[lang], vectorized_doc))
        except KeyError:
            print(f"Unkown language {lang} at index {i}. Known languages are: {lang_map.keys()}")
    print(f"{len(obs)} test observations.")
    return obs

## Classification Function

We also need to define the softmax function.

Softmax is technically

$$\text{softmax}(z_i) = \frac{e^{z_i}}{\sum^K_{j=1} e^{z_j}}$$

However, if implemented naïvely, this is not numerically stable. Instead, we use:

$$\text{softmax}(z_i) = \frac{e^{z_i - \text{max}(z)}}{\sum^K_{j=1} e^{z_j - \text{max}(z)}}$$


In [23]:
def softmax(z: npt.ArrayLike) -> npt.ArrayLike:
    """Compute the softmax of a vector `z`"""
    # exp(z) can get very large. For numerical stability, we subtract a vector of very large values (np.max(z)) from z.
    return np.exp(z - np.max(z)) / np.exp(z - np.max(z)).sum()

# Training the Classifier

### Compute the gradient

The formula for computing one element in our gradient is as follows (the partial derivitive of the negative log likelihood loss):

$$\frac{\partial L_{CE}}{\partial \mathbf{w}_{k,i}}=-(\mathbf{y}_k-\hat{\mathrm{y}}_k)\mathbf{x}_i$$

where $k$ is the **class** (rows of the matrix $\mathbf{w}$) and $i$ corresponds the the feature (columns of the matrix $\mathbf{w}$).

We will define a function `grad` for computing the whole gradient, a $K \times N$ matrix.

**This is the first piece of code that you'll write.**

In [24]:
def grad(W: np.ndarray, y: np.ndarray, x: np.ndarray) -> np.ndarray:
    """
    Calculates the gradient of the negative log likelihood loss, a [K * N] matrix, with respect to W.

    Parameters
    ----------
    W : np.ndarray
       A [K * N] matrix of weights, where K is the number of classes and N is the number of features.
    y : np.ndarray
       The true label of the observation, expressed as a [K * 1] one-hot vector.
    x : np.ndarray
       A [N * 1] vector of features.

    Returns
    -------
    np.ndarray
        The gradient of the loss with respect to W.
    """
    
    # Step 1: Compute the prediction (Softmax)
    z = W @ x  # Linear transformation (K * 1)
    exp_z = np.exp(z - np.max(z))  # Subtracting max for numerical stability
    softmax = exp_z / np.sum(exp_z)
    
    # Step 2: Compute the gradient (y - softmax) * x^T
    gradient = np.outer(softmax - y, x)  # (K * 1) - (K * 1), and x is (N * 1)
    
    return gradient

## Training Loop

Iterate over the observations in the training set $n$ times (in random order). For each item, compute the one-hot vector $\mathbf{y}$ and the probability distribution $\hat{\mathbf{y}}$. Use these values to compute the gradient. Update the parameters based on the gradient. At the end of each epoch (pass through the training data), compute the true positives, false positives, and false negatives for each target class based on the current weights, and report micro-averaged precision and recall.

How you report the metrics is up to you — we will not look at what you output to STDOUT — but it is important that you do this. **Otherwise, you will not be able to determine whether your model is training.**

You can also output the loss at each step (very noisy!), each epoch, or run the classifier on the dev set and report the metrics at the end of each epoch.

Remember that the 0th column in $W$ contains the biases. You will have to insert a $1$ at the beginning of the feature vector $x$ in order to accomodate this.

In [25]:
def train(observations: list, eta: float, epochs: int = 1) -> np.ndarray:
    """
    Given a set of observations, returns a trained multinomial logistic regression (softmax regression) model.

    Parameters
    ----------
    observations : list of tuples
        List of tuples where each tuple contains (label, features).
    eta : float
        The learning rate for gradient descent.
    epochs : int, optional (default=1)
        The number of epochs to train the model.

    Returns
    -------
    np.ndarray
        The trained model as a [K * N] weight matrix.
    """

    # Extract labels (y) and features (X) from the observations
    y = np.array([obs[0] for obs in observations])  # Extract labels
    X = np.array([obs[1] for obs in observations])  # Extract features

    # Get the number of samples and number of features
    num_samples, num_features = X.shape
    num_classes = y.shape[1]  # Assuming one-hot encoding for labels

    # Initialize weights randomly (shape: [K * N])
    W = np.random.randn(num_classes, num_features)

    # Training loop
    for epoch in range(epochs):
        total_loss = 0
        
        for i in range(num_samples):
            x_i = X[i]           # Feature vector for sample i (shape: [N])
            y_i = y[i]           # True label for sample i (one-hot vector, shape: [K])
            
            # Compute prediction (softmax)
            z = W @ x_i          # Linear transformation (shape: [K])
            exp_z = np.exp(z - np.max(z))  # Stability trick
            softmax = exp_z / np.sum(exp_z)  # Softmax output (shape: [K])
            
            # Compute the loss (negative log likelihood)
            loss = -np.sum(y_i * np.log(softmax + 1e-8))  # Adding a small value to avoid log(0)
            total_loss += loss
            
            # Compute gradient of the loss with respect to W
            grad_W = np.outer(softmax - y_i, x_i)  # Gradient for this sample (shape: [K * N])
            
            # Update weights using gradient descent
            W -= eta * grad_W
        
        # Print the loss for the current epoch (optional)
        print(f"Epoch {epoch+1}/{epochs}, Loss: {total_loss/num_samples:.4f}")
    
    return W

# Classification

The classification function is very simple.

In [26]:
def classify(W: np.ndarray, x: np.ndarray) -> np.intp:
    """
    Return the index of the hypothesized language (or class).

    Parameters
    ----------
    W : np.ndarray
        Weight matrix (one row for each category/language, one column for each feature).
    x : np.ndarray
        Vector of real-valued features.

    Returns
    -------
    np.intp
        The index of the hypothesized language (or class).
    """
    
    # Step 1: Compute the linear transformation
    z = W @ x  # z is the scores for each class (shape: [K])
    
    # Step 2: Find the index of the class with the highest score
    predicted_class = np.argmax(z)  # Index of the max value in z
    
    return predicted_class

# Evaluate the Classifier

Then, a function to train and evaluate the classifier.

In [27]:
def evaluate(train_set, test_set, eta, epochs=3):
    """Trains and evaluates a model."""
    print("Training model")
    W = train(train_set, eta, epochs=epochs)
    print("\nCLASSIFY")
    tp, fp, fn = Counter(), Counter(), Counter()
    for ref_lang_vec, x in test_set:
        ref_lang = np.argmax(ref_lang_vec)

        hyp_lang = classify(W, x)
        if hyp_lang == ref_lang:
            tp[ref_lang] += 1
        else:
            fp[hyp_lang] += 1
            fn[ref_lang] += 1
    # Print metrics

    test_macro_f1 = macro_f1(tp, fp, fn)
    test_micro_f1 = micro_f1(tp, fp, fn)
    print(f"macro-averaged F1:\t\t{test_macro_f1:.3f}")
    print(f"micro-averaged F1:\t\t{test_micro_f1:.3f}")
    return test_macro_f1, test_micro_f1

# Putting it all together

Load the data:

In [28]:
train_observations = load_data("train.tsv")
test_observations = load_data("test.tsv")

## Inspecting the data

Before we actually train the classifier, it's important to look at your data, and check that any assumptions you're making about it are justified. It's always useful at this point to check basic things, like:
- How many instances of train and test data do you have?
- What labels are in your data, and do those match between the train and test splits?
- What is the class balance (i.e. how many instances of each class) in your dataset? Is it balanced or unbalanced?

Output a dictionary for the train and test data that maps each label to the count of instances that have that label, e.g.: 
```
{"hausa": 4000, "indonesian":...}
```

Your dictionary should be sorted in descending order of occurrence of languages.

In [29]:
# this cell's output will be used for test 1.1
# your code here - train set

import csv
from collections import Counter

def load_labels_from_tsv(file_path: str) -> list:
    """
    Loads the labels (languages) from the first column of a TSV file.

    Parameters
    ----------
    file_path : str
        The path to the TSV file.

    Returns
    -------
    list
        A list of language labels from the first column.
    """
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            labels.append(row[0])  # The first column is the language label
    return labels

# Load the labels from the train.tsv file
train_labels = load_labels_from_tsv("train.tsv")

# Count the occurrences of each label
train_label_counts = Counter(train_labels)

# Sort the label counts in descending order
sorted_train_label_counts = dict(sorted(train_label_counts.items(), key=lambda item: item[1], reverse=True))

# Output the train set label counts
print("Train Set Label Counts:", sorted_train_label_counts)

Train Set Label Counts: {'swahili': 377, 'tagalog': 365, 'manobo': 111, 'hausa': 91, 'nahuatl': 90, 'indonesian': 86}


In [30]:
# this cell's output will be used for test 1.2
# your code here - test set

import csv
from collections import Counter

def load_labels_from_tsv(file_path: str) -> list:
    """
    Loads the labels (languages) from the first column of a TSV file.

    Parameters
    ----------
    file_path : str
        The path to the TSV file.

    Returns
    -------
    list
        A list of language labels from the first column.
    """
    labels = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            labels.append(row[0])  # The first column is the language label
    return labels

# Load the labels from the test.tsv file
test_labels = load_labels_from_tsv("test.tsv")

# Count the occurrences of each label
test_label_counts = Counter(test_labels)

# Sort the label counts in descending order
sorted_test_label_counts = dict(sorted(test_label_counts.items(), key=lambda item: item[1], reverse=True))

# Output the test set label counts
print("Test Set Label Counts:", sorted_test_label_counts)

Test Set Label Counts: {'swahili': 142, 'tagalog': 121, 'nahuatl': 36, 'manobo': 34, 'indonesian': 32, 'hausa': 28}


## Set hyperparameters and parameters

Before training the model, we have to set the learning rate $\eta$ and the order of the ngrams used in feature extraction.

In [31]:
eta = 0.0005  # Do not change this.
epochs = 4  # Do not change this.
order_of_ngrams = 1  # Do change this.

## Running our training and evaluation loops

Now train your classifier, and evaluate it on the test set. Vary the number of ngrams, and observe how it changes train and test F1.

In [32]:
train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=order_of_ngrams
)


test_set = preprocess_test_observations(test_observations, feature_map, lang_map, n=order_of_ngrams)


random.seed(27)
test_macro_f1, test_micro_f1 = evaluate(train_set, test_set, eta, epochs=epochs)

1120 training observations.
393 test observations.
Training model
Epoch 1/4, Loss: 6.3406
Epoch 2/4, Loss: 2.7149
Epoch 3/4, Loss: 2.0378
Epoch 4/4, Loss: 1.6915

CLASSIFY
macro-averaged F1:		0.697
micro-averaged F1:		0.733


Print the tuple of the best values of `(macro_f1, micro_f1)` for the model evaluated on your test set while varying the order of the ngrams. What value of n produced the best result? Why might that be?

In [33]:
# the output of this cell will be used for test 2.1
print((test_macro_f1, test_micro_f1))

(0.6971469989725778, 0.7328244274809159)


# Inspecting Classification Results

We've trained our classifier, and your final F1 should be pretty close to 1.0. Great job! But what does that mean for the languages you're actually classifying? Let's rewrite our evaluation code to allow us to look at our results instance-by-instance. For this, we're going to examine the results of a re-trained trigram classifier, trained for one epoch.

In [34]:
# Re-training a trigram classifier. Do not change this.

INSPECTION_NGRAMS = 3

train_set, lang_map, feature_map = preprocess_training_observations(
    train_observations, n=INSPECTION_NGRAMS
)
test_set = preprocess_test_observations(
    test_observations, feature_map, lang_map, n=INSPECTION_NGRAMS
)

random.seed(27)
W_inspect = train(train_set, eta, epochs=1)

## evaluate it as before. Check that this looks the same!
tp, fp, fn = Counter(), Counter(), Counter()
for ref_lang_vec, x in test_set:
    ref_lang = np.argmax(ref_lang_vec)

    hyp_lang = classify(W_inspect, x)
    if hyp_lang == ref_lang:
        tp[ref_lang] += 1
    else:
        fp[hyp_lang] += 1
        fn[ref_lang] += 1
# Print metrics

print(f"macro-averaged F1:\t\t{macro_f1(tp, fp, fn):.3f}")
print(f"micro-averaged F1:\t\t{micro_f1(tp, fp, fn):.3f}")

1120 training observations.
393 test observations.
Epoch 1/1, Loss: 9.6054
macro-averaged F1:		0.218
micro-averaged F1:		0.293


## Writing a prediction function

Write a function that takes a list of observations, and produces a list of either class indices, or class names based on a parameter. This will allow you both to look at individual results from evaluating on an existing set of data (like the test set), but also for you to evaluate your classifier on new data (i.e. any string). This will involve vectorizing the list of documents, and classifying those vectors. Once that's complete, construct an inverse language mapping, from predicted indices to the language they represent.

In [35]:
import numpy as np
import csv

def vectorize_document(document: str, feature_map: dict, ngram_length: int) -> np.ndarray:
    """
    Vectorizes a document by creating an n-gram based feature vector.

    Parameters
    ----------
    document : str
        The input document (a string).
    feature_map : dict
        A mapping from n-grams to feature indices.
    ngram_length : int
        The length of n-grams to extract.

    Returns
    -------
    np.ndarray
        The feature vector for the document.
    """
    vector = np.zeros(len(feature_map))

    # Extract n-grams from the document
    for i in range(len(document) - ngram_length + 1):
        ngram = document[i:i + ngram_length]
        if ngram in feature_map:
            vector[feature_map[ngram]] += 1
    
    return vector

def predict(
    documents: list[str],
    W: np.ndarray,
    feature_map: dict[str, int],
    lang_map: dict[str, np.ndarray],
    ngram_length: int,
    return_class_names=False,
):
    """
    Predicts the class indices or names for a list of documents.

    Parameters
    ----------
    documents : list of str
        List of input documents (strings).
    W : np.ndarray
        The weight matrix of the trained softmax classifier.
    feature_map : dict
        Mapping from n-grams to feature indices.
    lang_map : dict
        Mapping from language names to one-hot vectors.
    ngram_length : int
        The length of n-grams to extract from documents.
    return_class_names : bool, optional (default=False)
        If True, returns the predicted class names. Otherwise, returns class indices.

    Returns
    -------
    list
        List of predicted class indices or class names based on return_class_names.
    """
    
    # Inverse language mapping (from index to language name)
    inverse_lang_map = {np.argmax(v): k for k, v in lang_map.items()}
    
    predictions = []

    for doc in documents:
        # Step 1: Vectorize the document
        x = vectorize_document(doc, feature_map, ngram_length)
        
        # Step 2: Compute the prediction (softmax)
        z = W @ x
        exp_z = np.exp(z - np.max(z))  # Stability trick
        softmax = exp_z / np.sum(exp_z)
        
        # Step 3: Find the index of the predicted class
        predicted_index = np.argmax(softmax)
        
        # Step 4: Return class name or index
        if return_class_names:
            predictions.append(inverse_lang_map[predicted_index])
        else:
            predictions.append(predicted_index)
    
    return predictions

def load_data(file_path: str) -> list:
    """
    Loads data from a tsv file and returns a list of (label, document) tuples.

    Parameters
    ----------
    file_path : str
        The path to the tsv file.

    Returns
    -------
    list
        A list of (label, document) tuples.
    """
    observations = []
    with open(file_path, 'r', encoding='utf-8') as file:
        reader = csv.reader(file, delimiter='\t')
        for row in reader:
            label, document = row
            observations.append((label, document))
    return observations

Now, for each instance in the test set, print a tuple of the actual label, then the predicted label.

In [36]:
# Assuming we load the test data from a file
test_observations = load_data("test.tsv")  # Example function to load data

# the output of this cell will be used for test 3.1
test_labels, test_documents = zip(*test_observations)
test_predictions = predict(
    test_documents,
    W_inspect,
    feature_map,
    lang_map,
    ngram_length=INSPECTION_NGRAMS,
    return_class_names=True,
)

print(list(zip(test_labels, test_predictions)))

[('swahili', 'indonesian'), ('indonesian', 'hausa'), ('tagalog', 'tagalog'), ('indonesian', 'manobo'), ('hausa', 'manobo'), ('nahuatl', 'swahili'), ('hausa', 'swahili'), ('manobo', 'manobo'), ('swahili', 'swahili'), ('swahili', 'manobo'), ('tagalog', 'manobo'), ('indonesian', 'hausa'), ('tagalog', 'tagalog'), ('tagalog', 'tagalog'), ('swahili', 'swahili'), ('swahili', 'hausa'), ('hausa', 'hausa'), ('indonesian', 'manobo'), ('manobo', 'indonesian'), ('tagalog', 'indonesian'), ('swahili', 'manobo'), ('swahili', 'swahili'), ('manobo', 'tagalog'), ('swahili', 'tagalog'), ('manobo', 'indonesian'), ('swahili', 'tagalog'), ('tagalog', 'tagalog'), ('swahili', 'hausa'), ('swahili', 'manobo'), ('nahuatl', 'nahuatl'), ('nahuatl', 'manobo'), ('swahili', 'manobo'), ('swahili', 'swahili'), ('swahili', 'hausa'), ('tagalog', 'tagalog'), ('indonesian', 'manobo'), ('nahuatl', 'indonesian'), ('tagalog', 'manobo'), ('tagalog', 'tagalog'), ('tagalog', 'nahuatl'), ('tagalog', 'tagalog'), ('tagalog', 'tagalo

## Plotting a Confusion Matrix

 A _confusion matrix_ is a $k \times k$ matrix, where k is your number of classes, where the cell in position $(i,j)$ counts the number of instances that belong to class $i$ that were predicted to be in class $j$. The diagonal entries represent correct classifications; anything off of the diagonal represents an incorrect classification. The example below, from the [scikit learn documentation](https://scikit-learn.org/stable/modules/generated/sklearn.metrics.ConfusionMatrixDisplay.html), shows a confusion matrix for a binary classification problem. 

![A confusion matrix for a binary classification problem](confusion_matrix.png)


Confusion matrices can be useful to see what types of errors your classifier is making. If errors are concentrated into particular cells, it could indicate the kind of data that your classifier struggles with. Write a function to take a list of test observations, and output a numpy array that represents your classifier's confusion matrix.

In [44]:
import numpy as np

def get_confusion_matrix(
    labels: list[str], predictions: list[str], lang_map: dict[str, np.ndarray]
) -> np.ndarray:
    """
    Computes the confusion matrix for the given labels and predictions.

    Parameters
    ----------
    labels : list of str
        List of actual labels (true language classes).
    predictions : list of str
        List of predicted labels (predicted language classes).
    lang_map : dict
        Dictionary mapping language names to one-hot vectors.

    Returns
    -------
    np.ndarray
        A confusion matrix of shape (k, k), where k is the number of classes.
    """
    # Number of classes (languages)
    num_classes = len(lang_map)

    # Create an empty confusion matrix
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

    # Inverse mapping from one-hot vector to language index
    inverse_lang_map = {np.argmax(v): k for k, v in lang_map.items()}

    # Create a mapping from language names to indices
    lang_indices = {lang: idx for idx, lang in enumerate(lang_map.keys())}

    # Populate the confusion matrix
    for true_label, predicted_label in zip(labels, predictions):
        true_idx = lang_indices[true_label]         # Get the index of the true label
        predicted_idx = lang_indices[predicted_label]  # Get the index of the predicted label
        confusion_matrix[true_idx, predicted_idx] += 1  # Increment the corresponding cell

    return confusion_matrix

Now, print your confusion matrix. 

In [45]:
# the output of this cell will be used for test 3.2
get_confusion_matrix(test_labels, test_predictions, lang_map)

array([[ 2,  5,  6,  0, 11,  4],
       [ 7,  4,  9,  4,  4,  4],
       [ 3,  4,  5,  4,  3, 15],
       [ 3,  7,  6,  4, 15,  1],
       [48, 14, 19,  6, 37, 18],
       [ 4, 24,  4, 17,  9, 63]])

From your confusion matrix, how many Hausa examples are misclassified as Swahili? Print each of the misclassified documents on a new line. 

In [46]:
# Assuming lang_map is already defined as in previous examples
lang_indices = {lang: idx for idx, lang in enumerate(lang_map.keys())}

# Get the index for Hausa (actual class) and Swahili (predicted class)
hausa_idx = lang_indices['hausa']
swahili_idx = lang_indices['swahili']

# Step 1: Compute the confusion matrix
confusion_matrix = get_confusion_matrix(test_labels, test_predictions, lang_map)

# Step 2: Print the number of misclassified Hausa examples as Swahili
num_misclassified_hausa_as_swahili = confusion_matrix[hausa_idx, swahili_idx]
print(f"Number of Hausa examples misclassified as Swahili: {num_misclassified_hausa_as_swahili}")

# Step 3: Print the misclassified documents
print("Misclassified Hausa documents:")
for true_label, predicted_label, document in zip(test_labels, test_predictions, test_documents):
    if true_label == 'hausa' and predicted_label == 'swahili':
        print(document)

Number of Hausa examples misclassified as Swahili: 11
Misclassified Hausa documents:
Lokacin da mutanen da su ke kiwon aladun suka ga abin da ya faru, sai suka gudu! Suka ba da rahoton abin da ya faru ga mutanen cikin gari da na wannan kewayen.
Wannan shi ne dalilin shari'ar, domin haske ya zo duniya, amma mutane suka kaunaci duhu fiye da hasken, sabili da ayyukan su na mugunta ne.
Sai aka kawo kansa bisa tire, aka mika wa yarinyar, ta kuwa kai wa mahaifiyarta.
Ta wurin haka Yesu ya bada tabbatacce da kuma ingataccen alkwari.
Domin gaskiya ina gaya maku, har sama da kasa su shude, amma digo ko wasali daya ba za ya shude ba a cikin shari'ar, sai an cika dukan al'amura.
Sa'annan iklisiya dake dukan kasar Yahudiya, Galili da kuma Samariya suka sami salama da kuma ginuwa; suka kuwa ci gaba da tafiya cikin tsoron Ubangiji, da kuma ta'aziyar Ruhu Mai Tsarki, iklisiyar kuwa ta karu da yawan mutane.
Ya ce masu, "A rubuce yake, cewa Almasihu za ya sha wuya, zai tashi kuma daga matattu a rana ta

Can you formulate a hypothesis for why these Hausa examples are being misclassified as Swahili? Peruse the Wikipedia pages of the two languages, and inspect the data and features of the two languages. Consider reasons based in what you know about the languages, and about machine learning. Try to come up with 2-3 experiments you might run to validate your hypothesis.

## Examining Feature Weights

The classifier that we've built for softmax classification behaves in many ways like using several binary softmax classifiers stacked together. The $K \times N$ feature matrix can interpreted as a $1 \times N$ feature vector for each class. In this section, we'll examine the weights for a few of our classified languages. To get feature names out of these vectors, we'll construct an inverted feature map, that maps indices in the feature vectors back to n-grams. Then, extract the $1 \times N$ vector that corresponds to Hausa. Print the shape of the Hausa feature vector.

In [47]:
# the output of this cell will be used for test 4.1
# Invert the feature map
inverted_feature_map = {v: k for k, v in feature_map.items()}

# your code here

Now, let's find the features that are most strongly predictive of an instance being Hausa. From the Hausa feature vector, find the names of the features with the top-10 positive values. Print your results with a tuple of (feature name, feature_weight) on each line:
```
("aaa", 0.04597442104739769)
("bbb", 0.03454984682736487)
```

In [48]:
# Step 1: Invert the feature map
inverted_feature_map = {v: k for k, v in feature_map.items()}

# Step 2: Get the index for Hausa in the lang_map
hausa_index = np.argmax(lang_map['hausa'])

# Step 3: Extract the Hausa feature vector from the weight matrix W_inspect
hausa_feature_vector = W_inspect[hausa_index]

# Step 4: Print the shape of the Hausa feature vector
print("Hausa Feature Vector Shape:", hausa_feature_vector.shape)


Hausa Feature Vector Shape: (7941,)


Now, for each language, print the top-10 features. Print the name of each language, and then a list of it's top-10 features on the following line, e.g.

```
hausa
[("aaa", 0.04597442104739769)...]
indonesian
[("aaa", 0.04597442104739769)...]
```

In [49]:
# Step 1: Invert the feature map (already done in previous cells)
inverted_feature_map = {v: k for k, v in feature_map.items()}

# Step 2: Iterate through each language in the lang_map
for lang, one_hot_vec in lang_map.items():
    
    # Step 3: Get the index of the language in the weight matrix
    lang_index = np.argmax(one_hot_vec)
    
    # Step 4: Extract the feature vector for the language from the weight matrix W_inspect
    feature_vector = W_inspect[lang_index]
    
    # Step 5: Get the indices of the top 10 features (sorted by feature importance)
    top_10_feature_indices = np.argsort(feature_vector)[-10:][::-1]  # Sort in descending order
    
    # Step 6: Get the top 10 n-grams and their corresponding weights
    top_10_features = [(inverted_feature_map[idx], feature_vector[idx]) for idx in top_10_feature_indices]
    
    # Step 7: Print the language name and its top-10 features
    print(lang)
    print(top_10_features)

hausa
[('tod', 3.646713273175252), ('t.\n', 3.5816502349736314), ('gva', 3.464785690623641), ('zo ', 3.459363955105436), ('ife', 3.3587871235500426), ('r-m', 3.2774736330916565), ('sip', 3.19592774908268), ('gup', 3.1950040129726553), ('g F', 3.113817370345581), ('g-l', 3.09750009884081)]
indonesian
[('rek', 3.5191118566068207), ('abe', 3.505960683174251), ('ioi', 3.2142693863097693), ('Huf', 3.13780221163755), ('r n', 3.137749347063636), ('ahO', 3.109421152855386), ('iwe', 3.0987932683152586), ('\to ', 3.095725230014659), ('b d', 3.054443705516728), ('den', 3.0192616540537025)]
manobo
[('iok', 3.739655242987972), ('“Wa', 3.496477252822731), ('e j', 3.3231564585068383), ('Bum', 3.3216921501354784), ('Sek', 3.2872546068788004), ('wda', 3.2708411771800976), ('y s', 3.2414787221666943), ('ehi', 3.2004776865752507), ('rpi', 3.1848251834993015), ('Kud', 3.1435733557055667)]
nahuatl
[('u?”', 3.5861123394136967), ('zao', 3.546088757018726), ('t H', 3.2907012834909937), ('p. ', 3.2347044501484

Choose one of the languages, and peruse its Wikipedia page or other reliable resources to learn a bit more about the language. Do the top 10 features in that language make sense given what you've learned about the structure of the language? Why or why not? Again, consider reasons stemming both from what you've learned about the language, as well as machine learning and multinomial logistic regression specifically. 

## Extra Credit

Implement at least one of the experiments you devised to test your hypothesis regarding misclassification of Hausa as Swahili. In order to get credit you must submit not just the code and results, but also clearly describe your hypothesis, the experiment, and why the experiment is suitable to test your hypothesis. Full credit will be given to hypotheses and experiments that are well thought out, explained clearly and convincingly, and backed up with suitable evidence (computed or cited).

In [50]:
from collections import Counter

def extract_top_ngrams(documents: list[str], n: int, top_k: int = 10) -> list[tuple[str, int]]:
    """
    Extracts the most frequent n-grams from a list of documents.
    
    Parameters
    ----------
    documents : list of str
        List of documents from which to extract n-grams.
    n : int
        Length of n-grams to extract.
    top_k : int
        Number of top n-grams to return.
    
    Returns
    -------
    list of tuples
        A list of the top_k most frequent n-grams and their counts.
    """
    ngram_counter = Counter()
    
    # Iterate through each document and extract n-grams
    for doc in documents:
        ngrams = extract_ngrams(doc, n)  # Assuming extract_ngrams is defined elsewhere
        ngram_counter.update(ngrams)
    
    # Return the top_k most common n-grams
    return ngram_counter.most_common(top_k)

# Swahili와 Hausa 문서에서 상위 n-gram 추출
n = 3  # Trigram 사용

# Swahili와 Hausa 문서 필터링
hausa_documents = [doc for label, doc in train_observations if label == 'hausa']
swahili_documents = [doc for label, doc in train_observations if label == 'swahili']

# 상위 n-gram 추출
top_hausa_ngrams = extract_top_ngrams(hausa_documents, n=n, top_k=10)
top_swahili_ngrams = extract_top_ngrams(swahili_documents, n=n, top_k=10)

# 결과 출력
print("Top 10 Hausa n-grams:", top_hausa_ngrams)
print("Top 10 Swahili n-grams:", top_swahili_ngrams)

# Swahili와 Hausa의 공통 n-gram 확인
hausa_ngram_set = set([ngram for ngram, _ in top_hausa_ngrams])
swahili_ngram_set = set([ngram for ngram, _ in top_swahili_ngrams])
overlap = hausa_ngram_set & swahili_ngram_set

print("Overlapping n-grams between Hausa and Swahili:", overlap)
print("Number of overlapping n-grams:", len(overlap))

Top 10 Hausa n-grams: [('da ', 176), (' da', 163), (' ya', 116), ('a k', 111), ('in ', 107), ('ya ', 105), ('an ', 104), (' ka', 104), ('ka ', 94), ('a s', 82)]
Top 10 Swahili n-grams: [('wa ', 832), (' wa', 765), ('a k', 715), ('na ', 667), (' na', 555), ('a m', 525), (' ya', 469), (' ku', 448), ('ya ', 384), (' kw', 351)]
Overlapping n-grams between Hausa and Swahili: {'ya ', 'a k', ' ya'}
Number of overlapping n-grams: 3


## Hypothesis:
The reason Hausa is frequently misclassified as Swahili may be due to shared n-gram patterns between the two languages. Historically, both languages have undergone significant linguistic borrowing, particularly from languages like Arabic and English. This overlap could lead to the classifier being confused, as the model might learn similar feature representations for both languages.

## Experiment Process:
To test this hypothesis, I will run an experiment focused on analyzing the n-gram overlap between Swahili and Hausa. The steps are as follows:

1. Extract Top n-grams: First, I'll extract the top 10 most frequent n-grams from the training data for both Swahili and Hausa.

2. Analyze Common n-grams: I'll then compare the n-grams from both languages to identify how many of these n-grams are shared between the two languages.

3. Interpret the Results: If we find significant overlap in the n-grams between Swahili and Hausa, this would support the hypothesis that the model struggles to differentiate between the two due to shared features.

## Suitability of the Experiment:
This experiment is suitable for testing the hypothesis because:

1. Linguistic Similarity: Both Swahili and Hausa have historically borrowed words from common sources, like Arabic, so it’s reasonable to expect some overlap in n-gram patterns. By quantifying this overlap, we can test if this linguistic similarity contributes to the classifier’s confusion.

2. Model Behavior: In multinomial logistic regression, n-grams are treated as features that help classify languages. If Swahili and Hausa share a large number of top n-grams, it would make it harder for the model to draw distinct decision boundaries. Thus, analyzing n-gram overlap directly helps explain the model's behavior and why misclassifications might occur.